# Exercises

Continue your exploration of joins & set operators with these exercises:

1. According to the census population estimates, which county had the greatest percentage loss of population between 2010 & 2019? Try an internet search to find out what happened.
2. Apply the concepts you learned about `UNION` to create query results that merge queries of the census county population estimates for 2010 & 2019. Your results should includea column called `year` that specifies the year of the estimate for each row in the results.
3. Using the `percentile_cont()` function, determine the median percent change in estimated county population between 2010 & 2019.

---

# 1.

```
SELECT c2019.county_name,
	   c2019.state_name,
	   c2019.pop_est_2019,
	   c2010.estimates_base_2010,
	   c2019.pop_est_2019 - c2010.estimates_base_2010
	       AS difference,
	   round((c2019.pop_est_2019::numeric - 
	   	   c2010.estimates_base_2010) / 
		   c2010.estimates_base_2010 * 100, 1)
		   AS pct_change
FROM us_counties_pop_est_2019 AS c2019
LEFT JOIN us_counties_pop_est_2010 AS c2010
ON c2019.state_name = c2010.state_name
	AND c2019.county_name = c2010.county_name
ORDER BY pct_change;
```

Concho County in Texas saw the greatest percentage loss in population from 2010 to 2019. Concho County is already a small town, but it also was the home to Eden's prison. The prison once hosted more than 1,500 prisoners & employed about 250 people, closed in 2017, which could explain the steep drop-off in population.

# 2.

```
SELECT '2019' AS year,
	   county_name,
	   state_name,
	   pop_est_2019 AS county_population
FROM us_counties_pop_est_2019
UNION
SELECT '2010' AS year,
	   county_name,
	   state_name,
	   estimates_base_2010 AS county_population
FROM us_counties_pop_est_2010 AS c2010
ORDER BY state_name, county_name, year;
```

Our result has 6,284 rows, which is double of 3,142, because there are two years, 2010 & 2019, in the dataset.

<img src = "Exercise Images/Combining Census Data with UNION.png" width = "600" style = "margin:auto"/>

# 3.

```
SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY 
		   c2019.pop_est_2019) AS median_county_pop_2019,
	   percentile_cont(0.5) WITHIN GROUP (ORDER BY
	       c2010.estimates_base_2010)
		   AS median_county_pop_2010,
	   (percentile_cont(0.5) WITHIN GROUP (ORDER BY 
	   	   c2019.pop_est_2019) - 
		   percentile_cont(0.5) WITHIN GROUP (ORDER BY
	       c2010.estimates_base_2010)) / 
		   percentile_cont(0.5) WITHIN GROUP (ORDER BY 
		   c2019.pop_est_2019) * 100 AS pct_change 
FROM us_counties_pop_est_2019 AS c2019
LEFT JOIN us_counties_pop_est_2010 AS c2010
ON c2019.county_name = c2010.county_name
	AND c2019.state_name = c2010.state_name;
```

<img src = "Exercise Images/Median County Population Change from 2010 to 2019.png" width = "600" style = "margin:auto"/>

The median county population decreased from 2010 to 2019, but the difference is miniscule: -0.51%.